In [9]:
import numpy as np
import pandas as pd
import json
import requests
import lxml.html as lh

from urllib.request import urlopen

In [5]:
semester = 'SP22'
baseURL = f'https://classes.cornell.edu/api/2.0/search/classes.json?roster={semester}&subject=ECE'

response = urlopen(baseURL)
classData = json.loads(response.read())['data']['classes']

for class_ in classData:
    print(class_)

In [69]:
categories = ['CA', 'HA', 'KCM', 'LA', 'SBA', 'CE', 'ALC', 'SCD', 'HST', 'ETM', 'SSC', 'GLC']
completeData = []

for category in categories:

    LSURL = f'https://apps.engineering.cornell.edu/liberalstudies/{category}.cfm'
    response = requests.get(LSURL)
    data = lh.fromstring(response.content)
    tableData = data.xpath('//tr')

    headers = ['Category', 'Department + Course Number', 'Distributions']

    for row in tableData[1:]:

        rowElements = list(row.iterchildren())

        distributions = rowElements[3].text_content().split(', ')
        distributionsFormatted = []

        for distribution in distributions:
            newStr = distribution.replace('(', '').replace(')', '')
            distributionsFormatted.append(newStr)


        completeData.append([ category, f'{rowElements[0].text_content()} {rowElements[1].text_content()}', distributionsFormatted])

In [72]:
df = pd.DataFrame(completeData)
df.columns = headers
df.tail()

,Category,Department + Course Number,Distributions
7355,GLC,STS 4634,"[GLC-AS, HST-AS]"
7356,GLC,STS 4691,"[PBS-AS, BIO-AS, GLC-AS]"
7357,GLC,STS 4721,"[CA-AS, GLC-AS, SSC-AS]"
7358,GLC,VISST 3227,"[LA-AS, GLC-AS]"
7359,GLC,VISST 3581,"[CA-AS, ALC-AS, GLC-AS]"


In [73]:
df.to_csv('liberal_studies.csv', index = None)